## Mars Photogrammetry Preprocessing Pipeline

By Christian Tate, Cornell University; Ithaca, NY

https://github.com/cdt59/MPPP


In [1]:
# Import python modules

import numpy as np
import cv2, glob, time, os
import matplotlib.pyplot as plt

# from planetaryimage import PDS3Image
# import colour_demosaicing
# from PIL import Image
# import matplotlib.cm as cm
# from scipy import interpolate
# from scipy.spatial.transform import Rotation as R
# import colour_demosaicing
import pandas as pd

%run MPPP.py

%matplotlib inline

# time.sleep(20*60)

In [8]:
# Set input path to IMG files and search criteria

pre = ''
suf = 'references'


# Set input and output directories
directory_input  = 'Z:/Mastcam-Z/agisoft/data'
directory_output = 'Z:/Mastcam-Z/agisoft/images'

# directory_input  = 'C:/Users/cdt59/Desktop/heli/data'
# directory_output = 'C:/Users/cdt59/Desktop/heli'

# directory_input  = 'C:/Users/cdt59/Desktop/agisoft/data'
# directory_output = 'C:/Users/cdt59/Desktop/agisoft/images'

# directory_input  = 'C:/Users/cdt59/Downloads/Mars_2020_sol_681/data'
# directory_output = directory_input
# sol = 6

sol = '7'

pre = 'Z*0_0'+str(sol)+'*_07'
suf = 'refs_'+str(sol).zfill(3)+'_zcam'

# pre = '*F_0'+str(sol)+'*_07'
# suf = 'refs_'+str(sol).zfill(3)+'_ecam'

# pre = '*M_0'+str(sol)+'*_07'
# suf = 'refs_'+str(sol).zfill(3)+'_vce_ecam'

# pre = '*V0'+str(sol)+'*_07'
# suf = 'refs_'+str(sol).zfill(3)+'_vec'

# pre = 'H*_0'+str(sol)
# suf = 'refs_'+str(sol).zfill(3)+'_heli'


sequ_id = ''

path_input     = directory_input
IMG_paths      = [] 
IMG_paths     += sorted( glob.glob(  directory_input + '/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 
IMG_paths     += sorted( glob.glob(  directory_input + '/*/ids/rdr/*/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 

if 'F_0' in pre or 'V' in pre or 'M' in pre: 
    IMG_paths += sorted( glob.glob(  directory_input + '/datadrive/*/ids/rdr/*/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 
    IMG_paths += sorted( glob.glob(  directory_input + '/datadrive/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 
if 'Z' in pre: 
    IMG_paths += sorted( glob.glob(  directory_input + '/zcam/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 
    IMG_paths += sorted( glob.glob(  directory_input + '/zcam/*/' +pre+'*_N*'+sequ_id+'*.IMG' ) ) 


IMG_paths = IMG_paths[:]



print( len(IMG_paths), 'images\n')

# IMG_paths = [ 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0600s\\ZL0_0690_0728196929_035RAD_N0321184ZCAM08692_0340LMA01.IMG','Z:/Mastcam-Z/agisoft/data/zcam\\sol_0600s\\ZL0_0690_0728198458_034RAD_N0321184ZCAM08692_0340LMA01.IMG' ]

# # remove duplicates in image set

# %run MPPP.pya 

IMG_paths

10698 images



['Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729082888_303RAD_N0330262ZCAM08698_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729082952_364RAD_N0330262ZCAM03538_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729083334_081RAD_N0330262ZCAM03014_048085A01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093730_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093868_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093893_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093909_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093942_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\ZL0_0700_0729093958_894RAD_N0330366ZCAM08699_1100LMA01.IMG',
 'Z:/Mastcam-Z/agisoft/data/zcam\\sol_0700s\\Z

In [ ]:
%%time

# time.sleep(1000)

%run MPPP.py

# image_list_process( IMG_paths[::-1], directory_output, suf, find_offsets_mode = 1 )
# image_list_process( IMG_paths[:], directory_output, suf, find_offsets_mode = 0, frame='rnav' )
image_list_process( IMG_paths[:], directory_output, suf, find_offsets_mode = 0, frame='site3' )

10698 images

0 ZL0_0700_0729082888_303RAD_N0330262ZCAM08698_1100LMA01.IMG
sol 700 site 33 drive 262  zenith angle -699 scale 0.90
XYZ_ENU = [-2375.50, 790.76, 40.63] YPR = [33, 78, 72]  OPK = [95, 39, 9]

1 ZL0_0700_0729082952_364RAD_N0330262ZCAM03538_1100LMA01.IMG
sol 700 site 33 drive 262  zenith angle -664 scale 0.90
XYZ_ENU = [-2375.50, 790.76, 40.63] YPR = [-127, 78, -89]  OPK = [90, 38, 12]

2 ZL0_0700_0729083334_081RAD_N0330262ZCAM03014_048085A01.IMG
resizing image size (688, 624, 3) by padding = [ left, right, top, bottom ] = [ 1024, 0, 0, 512 ]
sol 700 site 33 drive 262  zenith angle -2702 scale 0.90
XYZ_ENU = [-2375.26, 790.88, 40.65] YPR = [143, 43, 16]  OPK = [-26, -38, -146]

3 ZL0_0700_0729093730_894RAD_N0330366ZCAM08699_1100LMA01.IMG
sol 700 site 33 drive 366  zenith angle -1882 scale 0.61
XYZ_ENU = [-2367.62, 799.07, 43.81] YPR = [0, 57, -2]  OPK = [57, -3, -0]

4 ZL0_0700_0729093868_894RAD_N0330366ZCAM08699_1100LMA01.IMG
sol 700 site 33 drive 366  zenith angle -1927 s

41 ZL0_0702_0729253278_614RAD_N0330366ZCAM08700_1100LMA02.IMG
sol 702 site 33 drive 366  zenith angle -551 scale 0.81
XYZ_ENU = [-2367.39, 798.93, 43.82] YPR = [288, 80, 79]  OPK = [-92, 29, -170]

42 ZL0_0702_0729253326_645RAD_N0330366ZCAM08700_1100LMA02.IMG
sol 702 site 33 drive 366  zenith angle -642 scale 0.81
XYZ_ENU = [-2367.40, 798.92, 43.81] YPR = [294, 79, 80]  OPK = [-92, 34, -168]

43 ZL0_0702_0729253344_613RAD_N0330366ZCAM08700_1100LMA02.IMG
sol 702 site 33 drive 366  zenith angle -727 scale 0.81
XYZ_ENU = [-2367.40, 798.91, 43.81] YPR = [300, 77, 81]  OPK = [-93, 39, -166]

44 ZL0_0702_0729253358_636RAD_N0330366ZCAM08700_1100LMA02.IMG
sol 702 site 33 drive 366  zenith angle -804 scale 0.81
XYZ_ENU = [-2367.42, 798.90, 43.81] YPR = [139, 76, -86]  OPK = [-89, 45, -167]

45 ZL0_0702_0729253372_614RAD_N0330366ZCAM08700_1100LMA01.IMG
sol 702 site 33 drive 366  zenith angle -813 scale 0.81
XYZ_ENU = [-2367.41, 798.90, 43.81] YPR = [304, 76, 80]  OPK = [-94, 44, -164]

46 ZL0_07

81 ZL0_0702_0729272945_318RAD_N0330402ZCAM08701_1100LMA01.IMG
sol 702 site 33 drive 402  zenith angle -2101 scale 0.54
XYZ_ENU = [-2371.86, 805.97, 45.41] YPR = [-35, 53, -6]  OPK = [50, 22, 21]

82 ZL0_0702_0729272965_318RAD_N0330402ZCAM08701_1100LMA01.IMG
sol 702 site 33 drive 402  zenith angle -2129 scale 0.54
XYZ_ENU = [-2371.86, 805.95, 45.41] YPR = [-43, 53, -8]  OPK = [48, 26, 27]

83 ZL0_0702_0729272979_318RAD_N0330402ZCAM08701_1100LMA01.IMG
sol 702 site 33 drive 402  zenith angle -2167 scale 0.53
XYZ_ENU = [-2371.86, 805.94, 45.41] YPR = [-51, 52, -10]  OPK = [46, 30, 34]

84 ZL0_0702_0729273014_318RAD_N0330402ZCAM08701_1100LMA01.IMG
sol 702 site 33 drive 402  zenith angle -2213 scale 0.53
XYZ_ENU = [-2371.85, 805.92, 45.40] YPR = [-59, 51, -11]  OPK = [42, 34, 40]

85 ZL0_0702_0729273034_318RAD_N0330402ZCAM08701_1100LMA01.IMG
sol 702 site 33 drive 402  zenith angle -2267 scale 0.53
XYZ_ENU = [-2371.84, 805.91, 45.40] YPR = [-67, 50, -13]  OPK = [38, 38, 48]

86 ZL0_0702_07292

119 ZL0_0706_0729631563_098RAD_N0330430ZCAM01025_034050A01.IMG
resizing image size (1200, 1600, 3) by padding = [ left, right, top, bottom ] = [ 32, 16, 0, 0 ]
sol 706 site 33 drive 430  zenith angle -3244 scale 0.34
XYZ_ENU = [-2371.68, 805.71, 45.40] YPR = [292, 33, 10]  OPK = [177, 34, -70]

120 ZL0_0706_0729631609_098RAD_N0330430ZCAM01025_034050A01.IMG
resizing image size (1200, 1600, 3) by padding = [ left, right, top, bottom ] = [ 32, 16, 0, 0 ]
sol 706 site 33 drive 430  zenith angle -996 scale 0.33
XYZ_ENU = [-2371.56, 805.88, 45.43] YPR = [159, 73, 357]  OPK = [-108, -17, -186]

121 ZL0_0706_0729631848_098RAD_N0330430ZCAM01025_034050A01.IMG
resizing image size (1200, 1600, 3) by padding = [ left, right, top, bottom ] = [ 32, 16, 0, 0 ]
sol 706 site 33 drive 430  zenith angle -1127 scale 0.32
XYZ_ENU = [-2371.65, 805.95, 45.45] YPR = [71, 70, 329]  OPK = [-143, -69, -241]

122 ZL0_0706_0729631858_098RAD_N0330430ZCAM01025_034050A01.IMG
resizing image size (1200, 1600, 3) by padd

158 ZL0_0707_0729718917_409RAD_N0330612ZCAM08708_1100LMA02.IMG
sol 707 site 33 drive 612  zenith angle -968 scale 0.42
XYZ_ENU = [-2386.18, 830.94, 49.13] YPR = [-136, 73, -32]  OPK = [-40, 71, -219]

159 ZL0_0707_0729718933_410RAD_N0330612ZCAM08708_1100LMA01.IMG
sol 707 site 33 drive 612  zenith angle -996 scale 0.42
XYZ_ENU = [-2386.17, 830.94, 49.13] YPR = [-139, 73, -30]  OPK = [-48, 67, -210]

160 ZL0_0707_0729718949_410RAD_N0330612ZCAM08708_1100LMA01.IMG
sol 707 site 33 drive 612  zenith angle -1021 scale 0.42
XYZ_ENU = [-2386.16, 830.95, 49.13] YPR = [-142, 72, -28]  OPK = [-54, 63, -204]

161 ZL0_0707_0729718982_410RAD_N0330612ZCAM08708_1100LMA01.IMG
sol 707 site 33 drive 612  zenith angle -1045 scale 0.41
XYZ_ENU = [-2386.15, 830.95, 49.13] YPR = [-145, 72, -26]  OPK = [-58, 58, -200]

162 ZL0_0707_0729719000_409RAD_N0330612ZCAM08708_1100LMA01.IMG
sol 707 site 33 drive 612  zenith angle -1067 scale 0.41
XYZ_ENU = [-2386.14, 830.95, 49.13] YPR = [-148, 71, -24]  OPK = [-61, 53,

...

405 ZL0_0709_0729888019_928RAD_N0332864ZCAM08711_1100LMA01.IMG
sol 709 site 33 drive 2864  zenith angle -2811 scale 0.80
XYZ_ENU = [-2696.63, 1010.55, 75.16] YPR = [-121, 41, -10]  OPK = [-14, 40, -237]

406 ZL0_0709_0729888037_928RAD_N0332864ZCAM08711_1100LMA01.IMG
sol 709 site 33 drive 2864  zenith angle -2858 scale 0.80
XYZ_ENU = [-2696.61, 1010.55, 75.16] YPR = [-128, 40, -10]  OPK = [-19, 37, -229]

407 ZL0_0709_0729888052_928RAD_N0332864ZCAM08711_1100LMA01.IMG
sol 709 site 33 drive 2864  zenith angle -2960 scale 0.80
XYZ_ENU = [-2696.66, 1010.55, 75.15] YPR = [-112, 38, -9]  OPK = [-5, 39, -250]

408 ZL0_0709_0729888096_928RAD_N0332864ZCAM08711_1100LMA01.IMG
sol 709 site 33 drive 2864  zenith angle -3063 scale 0.80
XYZ_ENU = [-2696.62, 1010.55, 75.15] YPR = [-128, 37, -9]  OPK = [-17, 34, -230]

409 ZL0_0709_0729888117_928RAD_N0332864ZCAM08711_1100LMA01.IMG
sol 709 site 33 drive 2864  zenith angle -3012 scale 0.80
XYZ_ENU = [-2696.64, 1010.55, 75.15] YPR = [-120, 37, -9]  OPK = [

446 ZL0_0710_0729980144_207RAD_N0332864ZCAM08712_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -1900 scale 0.68
XYZ_ENU = [-2696.73, 1010.60, 75.18] YPR = [-73, 57, -10]  OPK = [37, 48, 51]

447 ZL0_0710_0729980167_206RAD_N0332864ZCAM08712_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -1933 scale 0.68
XYZ_ENU = [-2696.72, 1010.59, 75.18] YPR = [-80, 56, -10]  OPK = [31, 51, 59]

448 ZL0_0710_0729980187_206RAD_N0332864ZCAM08712_1100LMA01.IMG
sol 710 site 33 drive 2864  zenith angle -1968 scale 0.68
XYZ_ENU = [-2696.71, 1010.58, 75.17] YPR = [-86, 56, -11]  OPK = [23, 53, 69]

449 ZL0_0710_0729980201_239RAD_N0332864ZCAM08712_1100LMA01.IMG
sol 710 site 33 drive 2864  zenith angle -2005 scale 0.67
XYZ_ENU = [-2696.70, 1010.57, 75.17] YPR = [-93, 55, -11]  OPK = [15, 54, 79]

450 ZL0_0710_0729980214_206RAD_N0332864ZCAM08712_1100LMA01.IMG
sol 710 site 33 drive 2864  zenith angle -2039 scale 0.67
XYZ_ENU = [-2696.68, 1010.56, 75.17] YPR = [-100, 54, -12]  OPK = [7, 55, -270]

487 ZL0_0710_0729980988_159RAD_N0332864ZCAM08713_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -399 scale 0.64
XYZ_ENU = [-2696.64, 1010.81, 75.22] YPR = [17, 83, 300]  OPK = [105, -77, 9]

488 ZL0_0710_0729981000_159RAD_N0332864ZCAM08713_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -398 scale 0.64
XYZ_ENU = [-2696.65, 1010.81, 75.22] YPR = [11, 83, 299]  OPK = [101, -71, 4]

489 ZL0_0710_0729981012_159RAD_N0332864ZCAM08713_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -406 scale 0.64
XYZ_ENU = [-2696.66, 1010.80, 75.22] YPR = [7, 83, 301]  OPK = [99, -66, 2]

490 ZL0_0710_0729981024_159RAD_N0332864ZCAM08713_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -418 scale 0.64
XYZ_ENU = [-2696.67, 1010.80, 75.22] YPR = [5, 83, 304]  OPK = [98, -61, 1]

491 ZL0_0710_0729981036_159RAD_N0332864ZCAM08713_1100LMA02.IMG
sol 710 site 33 drive 2864  zenith angle -411 scale 0.64
XYZ_ENU = [-2696.68, 1010.80, 75.22] YPR = [0, 83, 304]  OPK = [97, -56, 0]

492 ZL0_071

sol 711 site 33 drive 2864  zenith angle -338 scale 0.89
XYZ_ENU = [-2696.69, 1010.58, 75.20] YPR = [348, 84, 54]  OPK = [98, 65, -3]

528 ZL0_0711_0730055352_803RAD_N0332864ZCAM08714_1100LMA02.IMG
sol 711 site 33 drive 2864  zenith angle -356 scale 0.89
XYZ_ENU = [-2696.68, 1010.57, 75.20] YPR = [345, 84, 56]  OPK = [101, 70, -5]

529 ZL0_0711_0730055370_803RAD_N0332864ZCAM08714_1100LMA02.IMG
sol 711 site 33 drive 2864  zenith angle -380 scale 0.89
XYZ_ENU = [-2696.67, 1010.57, 75.20] YPR = [340, 83, 56]  OPK = [105, 75, -9]

530 ZL0_0711_0730055382_803RAD_N0332864ZCAM08714_1100LMA02.IMG
sol 711 site 33 drive 2864  zenith angle -416 scale 0.89
XYZ_ENU = [-2696.66, 1010.57, 75.20] YPR = [332, 83, 53]  OPK = [116, 80, -20]

531 ZL0_0711_0730055394_803RAD_N0332864ZCAM08714_1100LMA02.IMG
sol 711 site 33 drive 2864  zenith angle -423 scale 0.89
XYZ_ENU = [-2696.65, 1010.56, 75.20] YPR = [329, 83, 55]  OPK = [139, 84, -43]

532 ZL0_0711_0730055407_803RAD_N0332864ZCAM08714_1100LMA02.IMG
sol 

sol 711 site 33 drive 2864  zenith angle -264 scale 0.89
XYZ_ENU = [-2696.50, 1010.66, 75.20] YPR = [140, 85, -45]  OPK = [-87, 5, -177]

569 ZL0_0711_0730055875_803RAD_N0332864ZCAM08714_1100LMA01.IMG
sol 711 site 33 drive 2864  zenith angle -187 scale 0.89
XYZ_ENU = [-2696.50, 1010.67, 75.20] YPR = [267, 87, 82]  OPK = [-90, 5, -177]

570 ZL0_0711_0730055891_803RAD_N0332864ZCAM08714_1100LMA01.IMG
sol 711 site 33 drive 2864  zenith angle -157 scale 0.89
XYZ_ENU = [-2696.50, 1010.68, 75.21] YPR = [263, 87, 83]  OPK = [-90, -0, -177]

571 ZL0_0711_0730055904_803RAD_N0332864ZCAM08714_1100LMA01.IMG
sol 711 site 33 drive 2864  zenith angle -248 scale 0.89
XYZ_ENU = [-2696.50, 1010.67, 75.20] YPR = [140, 86, -40]  OPK = [-87, -0, -177]

572 ZL0_0711_0730055918_803RAD_N0332864ZCAM08714_1100LMA01.IMG
sol 711 site 33 drive 2864  zenith angle -433 scale 0.89
XYZ_ENU = [-2696.50, 1010.67, 75.20] YPR = [158, 82, -22]  OPK = [-83, -0, -177]

573 ZL0_0711_0730055930_803RAD_N0332864ZCAM08714_1100LMA0

In [10]:
%run MPPP.py

# im = image( 'C:/Users/cdt59/Desktop/heli/data//HSF_0174_0682390593_685RAD_N0120001HELI00005_000085J02.IMG' ) # (70,40,2)
# im = image( 'C:/Users/cdt59/Desktop/heli/data//00174//ids//rdr//heli//HNM_0174_0682390574_405RAD_N0120001HELI02729_0000LUJ02.IMG', ) # (140,0,0)
im = image( 'C:/Users/cdt59/Desktop/agisoft/data\\00784\\ids\\rdr\\ncam\\NLF_0784_0736548853_487RAD_N0390926NCAM03784_0A0195J01.IMG' )
im = image( 'C:/Users/cdt59/Desktop/agisoft/data\\00784\\ids\\rdr\\ncam\\NRF_0784_0736548853_487RAD_N0390926NCAM03784_0A0195J01.IMG' )
# im = image( 'C:/Users/cdt59/Desktop/agisoft/data/zcam\\ZL0_0786_0736703534_098RAD_N0390926ZCAM08811_1100LMA01.IMG' ) 
im = image( 'C:/Users/cdt59/Desktop/agisoft/data/zcam\\ZL0_0792_0737263387_769RAD_N0390926ZCAM01023_034050A01.IMG' ) 
im = image( 'C:/Users/cdt59/Desktop/agisoft/data/zcam\\ZR0_0792_0737263387_769RAD_N0390926ZCAM01023_034050A01.IMG' ) 

im = image( IMG_paths[0] )

im.image_reference()


im.f, im.b1, im.b2, '', im.cxp, im.cyp, '', im.k1, im.k2, im.theta_degrees, im.filename

(739.45960558367506,
 0.16674927592828226,
 0.12914999846960368,
 '',
 8.2596623355000247,
 5.2848627656929921,
 '',
 0.050139599999999999,
 -0.017125399999999999,
 -89.989995286521903,
 'NLF_0690_0728189040_534RZS_N0321184NCAM00501_0A0295J01.IMG')

<Figure size 640x480 with 0 Axes>

In [90]:
(im.label['RSM_ARTICULATION_STATE']['ARTICULATION_DEVICE_ANGLE'][0])

Quantity(value=3.15473, units='rad')

In [33]:

cmod_xml = "<?xml version=\"1.0\" encoding=\"UTF-8\"?><calibration>\n  <projection>{}</projection>\n  <width>{}</width>\n  <height>{}</height>\n  <f>{:0.6f}</f>\n  <cx>{:0.6f}</cx>\n  <cy>{:0.6f}</cy>\n  <k1>{:0.6f}</k1>\n  <k2>{:0.6f}</k2>\n  <p1>{:0.6f}</p1>\n  <p2>{:0.6f}</p2>\n  <k3>{:0.6f}</k3>\n  <k4>{:0.6f}</k4>\n  <b1>{:0.6f}</b1>\n  <b2>{:0.6f}</b2>\n</calibration>"\
            .format( im.projection, im.w, im.h, im.f, im.cxp, im.cyp, im.k1, im.k2, im.p1, im.p2, im.k3, im.k4, im.b1, im.b2 )

print(cmod_xml)

cmod_save_path = 'C:/Users/cdt59/Documents/MPPP_workspace/cmods/M2020_ZR0_034mm.xml'
with open(cmod_save_path, "w") as text_file:
    text_file.write( cmod_xml )

<?xml version="1.0" encoding="UTF-8"?><calibration>
  <projection>frame</projection>
  <width>2560</width>
  <height>960</height>
  <f>1475.234610</f>
  <cx>9.092042</cx>
  <cy>494.029161</cy>
  <k1>0.051719</k1>
  <k2>-0.018715</k2>
  <p1>0.000000</p1>
  <p2>0.000000</p2>
  <k3>0.000000</k3>
  <k4>0.000000</k4>
  <b1>0.338698</b1>
  <b2>0.142715</b2>
</calibration>


In [84]:
N = 3
D_min = 1.8
[ D_min*N/(N-n) for n in range(1,N)]

[2.7, 5.4]

In [174]:
w,h = [640,480]

# fisheye
im = image( 'C:/Users/cdt59/Desktop/heli/data//00174//ids//rdr//heli//HNM_0174_0682390574_405RAD_N0120001HELI02729_0000LUJ02.IMG', ) # (140,0,0)
Xp,Yp,Zp,yp,pp,rp = [-482.863524,-649.370648,10.150207,162.253878,-5.142142,-9.563955]

# im = image( 'C:/Users/cdt59/Desktop/heli/data//00174//ids//rdr//heli//HNM_0174_0682390575_302RAD_N0120001HELI02756_0000LUJ02.IMG', ) # (140,0,0)
# Xp,Yp,Zp,yp,pp,rp = [-479.242935,-648.062762,10.033978,161.940305,-9.974779,-10.268652] 

f, cxp, cyp, k1, k2, b1, b2 = [275.5,2.2541485333336087,11.026877077021922,-0.0086719433263928569,0.0031444070599917777,0,0]


In [176]:
im.image_reference()

ypr = np.array([yp,pp,rp])
R_site2cam = find_R_from_ypr( ypr )
# R_cam2ned  = R.from_matrix( [[0,-1,0],[1,0,0],[0,0,1]] )

R_cam2site = R_cam2ned.inv() * R_site2cam #R_veh2cam * R_veh2site.inv()

C = [ Yp, Xp, -Zp ]

cx = cxp+w/2
cy = cyp+h/2

K_cam = np.array([
            [ f+b1, b2, cx ],
            [    0,  f, cy ],
            [    0,  0,b 1 ], ])

HVA = np.matmul( K_cam, R_cam2site.as_matrix() )
H = HVA[0,:]
V = HVA[1,:]
A = HVA[2,:]
O = A
Ra = np.array([0,k1,k2])
E = np.array([0.0,0.0,0.0])

hp = ( H - cx* A ) / f
vp = ( V - cy* A ) / f

GEOMETRIC_CAMERA_MODEL = im.label['GEOMETRIC_CAMERA_MODEL']
GEOMETRIC_CAMERA_MODEL['REFERENCE_COORD_SYSTEM_NAME']  = 'SITE_FRAME'
GEOMETRIC_CAMERA_MODEL['REFERENCE_COORD_SYSTEM_INDEX'] = '3'

GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_1'] = list( C )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_2'] = list( A )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_3'] = list( H )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_4'] = list( V )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_5'] = list( O )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_6'] = list( Ra )
GEOMETRIC_CAMERA_MODEL['MODEL_COMPONENT_Y'] = list( E )

print( im.filename)

text = [im.filename, C[0], C[1], C[2], A[0], A[1], A[2],  H[0], H[1], H[2], V[0], V[1], V[2], O[0], O[1], O[2], Ra[0], Ra[1], Ra[2], E[0], E[1], E[2], ]

np.savetxt( im.filename[:-4]+'.cmod', text[:], delimiter=',', fmt='%s ' )

HNM_0174_0682390574_405RAD_N0120001HELI02729_0000LUJ02.IMG


In [16]:
# For find_offset_mode = 1

# # pos_lines  = []
# # rover_xyzs = []
# # im_xyzs    = []
# # sols       = []
# # rmcs = []
sites  = [ rmcs[i][0] for i in range(len(rmcs))[::-1] ]
drives = [ rmcs[i][1] for i in range(len(rmcs))[::-1] ]
Xs     = [ rover_xyzs[i][0] for i in range(len(rover_xyzs))[::-1] ]
Ys     = [ rover_xyzs[i][1] for i in range(len(rover_xyzs))[::-1] ]
Zs     = [ rover_xyzs[i][2] for i in range(len(rover_xyzs))[::-1] ]

table = np.stack( [sols[::-1], sites, drives, Xs, Ys, Zs], axis=1)
np.round( table, 4 )

np.savetxt( directory_output+"/offsets.csv", table, delimiter="\t")

NameError: name 'rmcs' is not defined

In [186]:
q_T_aim = R.from_euler( 'zyx', (70,38,4),degrees=True )

q_cam2enu = q_T_aim.inv() *q_cam * im.q_G * im.q_M *q_ned2enu

q_cam2enu.as_rotvec(degrees=1)

array([ -13.95403568, -122.26286237,   67.64833285])

In [6]:
import urllib.request, json 

'''

https://mars.nasa.gov/mmgis-maps/M20/Layers/json/M20_waypoints.json

https://docs.google.com/spreadsheets/d/1GSNJL-e5iROCaKUC4BXL_OA8hWy8vUdfCN7levRNkZA/edit#gid=1232397498

''' 
# path =  'C:/Users/cornell/Downloads/M20_waypoints.json'
# with open( path ) as f:
#     data = json.load(f)

with urllib.request.urlopen("https://mars.nasa.gov/mmgis-maps/M20/Layers/json/M20_waypoints.json") as url:
    data = json.loads(url.read().decode())

    
site3_E = data['features'][0]['properties']['easting']
site3_N = data['features'][0]['properties']['northing']
site3_A = data['features'][0]['properties']['elev_geoid']


for i in range(len(data['features']))[::-1]:
    
    print( data['features'][i]['properties']['sol'],'\t',
           data['features'][i]['properties']['site'], '\t',
           data['features'][i]['properties']['drive'],'\t',
           '0.0'+'\t','0.0'+'\t','0.0'+'\t',
           np.round( np.float64(data['features'][i]['properties']['easting'])- np.float64(site3_E), 3),'\t',
           np.round( np.float64(data['features'][i]['properties']['northing'])- np.float64(site3_N), 3),'\t',
           np.round( np.float64(data['features'][i]['properties']['elev_geoid'])- np.float64(site3_A), 3),'\t',
         )

848 	 41 	 188 	 0.0	 0.0	 0.0	 -4781.195 	 2180.602 	 154.716 	
843 	 41 	 0 	 0.0	 0.0	 0.0	 -4782.156 	 2182.185 	 154.706 	
843 	 40 	 1706 	 0.0	 0.0	 0.0	 -4787.905 	 2181.431 	 154.447 	
838 	 40 	 1608 	 0.0	 0.0	 0.0	 -4783.193 	 2171.856 	 154.351 	
836 	 40 	 992 	 0.0	 0.0	 0.0	 -4733.315 	 2245.44 	 153.561 	
821 	 40 	 132 	 0.0	 0.0	 0.0	 -4649.131 	 2273.826 	 149.121 	
821 	 40 	 70 	 0.0	 0.0	 0.0	 -4644.041 	 2275.221 	 148.731 	
818 	 40 	 0 	 0.0	 0.0	 0.0	 -4649.281 	 2276.728 	 148.901 	
818 	 39 	 1550 	 0.0	 0.0	 0.0	 -4648.226 	 2283.74 	 148.704 	
816 	 39 	 1418 	 0.0	 0.0	 0.0	 -4629.225 	 2300.703 	 147.718 	
816 	 39 	 1294 	 0.0	 0.0	 0.0	 -4625.095 	 2277.624 	 147.133 	
797 	 39 	 1170 	 0.0	 0.0	 0.0	 -4625.436 	 2254.732 	 148.838 	
795 	 39 	 1112 	 0.0	 0.0	 0.0	 -4623.796 	 2253.281 	 148.705 	
793 	 39 	 1036 	 0.0	 0.0	 0.0	 -4622.209 	 2248.958 	 148.201 	
784 	 39 	 926 	 0.0	 0.0	 0.0	 -4624.55 	 2258.333 	 148.341 	
784 	 39 	 858 	 0.0	 0.0

In [29]:
data['features'][-1]

{'type': 'Feature',
 'properties': {'RMC': '40_132',
  'site': 40,
  'drive': 132,
  'sol': 821,
  'easting': 4349844.955,
  'northing': 1095573.521,
  'elev_geoid': -2420.788818,
  'elev_radii': -4108.919434,
  'radius': 3392081.08,
  'lon': 77.36819431,
  'lat': 18.48298797,
  'roll': 8.657,
  'pitch': 4.009,
  'yaw': -106.695,
  'yaw_rad': -1.8622,
  'tilt': 9.534,
  'dist_m': 10.97,
  'dist_total_m': 18718.678,
  'dist_km': 18.72,
  'dist_mi': 11.63,
  'final': 'y',
  'Note': 'Used Mobility Report WID 48211',
  'images': [{'name': 'Panorama',
    'isPanoramic': True,
    'url': 'Layers/mosaics/N_LRGB_0821_RZS_0400132_CYL_L_AUTOGENJ01.jpg',
    'rows': '2247',
    'columns': '9292',
    'azmin': '0',
    'azmax': '360',
    'elmin': '-69.3863',
    'elmax': '17.6854',
    'elzero': '457.492'}]},
 'geometry': {'type': 'Point',
  'coordinates': [77.36819431, 18.48298797, -2420.788818]}}